Εισάγει τις απαραίτητες βιβλιοθήκες για το web scraping, την επεξεργασία κειμένου και τις λειτουργίες της μηχανής αναζήτησης.
- 'requests': για την υποβολή αιτημάτων HTTP για τη λήψη δεδομένων από το Βικιπαίδεια.
- 'pandas': για χειρισμό και αποθήκευση δεδομένων.
- 'nltk': για εργασίες επεξεργασίας φυσικής γλώσσας, όπως tokenization και αφαίρεση λέξεων-κλειδιών.
- 're': για κανονικές εκφράσεις για την επεξεργασία κειμένου.
- 'collections.defaultdict': για αποτελεσματική δημιουργία ανεστραμμένου δείκτη.
- 'sklearn': για την εφαρμογή της διανυσματοποίησης TF-IDF και τον υπολογισμό της ομοιότητας των συνημίτονων.

In [1]:
import requests
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Ορίζει μια λειτουργία ανάκτησης σελίδων της Βικιπαίδειας που ανήκουν σε μια συγκεκριμένη κατηγορία.
- Χρησιμοποιεί το API της Wikipedia για να πάρει σελίδες από τη δεδομένη κατηγορία.
- Περιορίζει τον αριθμό των σελίδων που ανακτώνται χρησιμοποιώντας την παράμετρο 'max_pages'.


In [2]:
def get_pages_from_category(category, max_pages=10):
    url = f"https://en.wikipedia.org/w/api.php?action=query&list=categorymembers&cmtitle=Category:{category}&cmlimit={max_pages}&format=json"
    response = requests.get(url).json()
    pages = response['query']['categorymembers']
    return [page['title'] for page in pages]

Ορίζει μια λειτουργία για την ανάκτηση του περιεχομένου μιας σελίδας Βικιπαίδεια, δεδομένου του τίτλου της.
- Ανακτά απλό περιεχόμενο κειμένου χρησιμοποιώντας το Wikipedia API.
- Χειρίζεται τις απαντήσεις JSON για να εξαγάγει το περιεχόμενο κειμένου της σελίδας.



In [3]:
def get_page_content(title):
    url = f"https://en.wikipedia.org/w/api.php?action=query&prop=extracts&explaintext&titles={title}&format=json"
    response = requests.get(url).json()
    pages = response['query']['pages']
    content = next(iter(pages.values())).get('extract', '')
    return content

Εξασφαλίζει ότι οι απαραίτητοι πόροι NLTK είναι κατεβασμένοι για tokenization και την αφαίρεση λέξεων-κλειδιών.
- Κατεβάζει το punt tokenizer για sentence splitting και tokenization.
- Κατεβάζει τη λίστα αγγλικών λέξεων-κλειδιών για να φιλτράρει κοινές λέξεις.


In [4]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sevak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sevak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Ορίζει μια λειτουργία προεπεξεργασίας κειμένου με special character removal, tokenizing και stemming.
- Μετατρέπει το κείμενο σε lowercase για να εξασφαλίσει το case insensitivity.
- Αφαιρεί μη αλφαριθμητικούς χαρακτήρες χρησιμοποιώντας κανονικές εκφράσεις.
- Μετατρέπει το κείμενο σε μεμονωμένες λέξεις.
- Αφαιρεί τις λέξεις-κλειδιά για να μειώσει το "θόρυβο" στα δεδομένα.
- Εφαρμόζεται για τη μείωση των λέξεων στις ριζικές τους μορφές για καλύτερη αντιστοίχιση.

In [5]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    ps = PorterStemmer()
    tokens = [ps.stem(word) for word in tokens]
    return tokens

Ορίζει μια συνάρτηση για τη δημιουργία ενός ανεστραμμένου ευρετηρίου για αποτελεσματικές λειτουργίες αναζήτησης.
- Χαρτογραφεί κάθε μοναδικό όρο σε μια λίστα αναγνωριστικών εγγράφων όπου εμφανίζεται.
- Επιτρέπει την ταχεία ανάκτηση εγγράφων που περιέχουν συγκεκριμένους όρους.

In [6]:
def create_inverted_index(documents):
    inverted_index = defaultdict(list)
    for doc_id, content in enumerate(documents):
        for term in set(content):
            inverted_index[term].append(doc_id)
    return inverted_index

Ορίζει μια συνάρτηση για την επεξεργασία ερωτημάτων boolean (AND, OR, NOT) χρησιμοποιώντας το αντίστροφο ευρετήριο.
- Υποστηρίζει συνδυασμό όρων αναζήτησης χρησιμοποιώντας λογικούς τελεστές.
- Εκτελεί λειτουργίες AND, OR και NOT για να βελτιώσει τα αποτελέσματα αναζήτησης.

In [7]:
def process_boolean_query(query, index):
    query_terms = re.findall(r'\w+', query.lower())
    operator = 'AND'
    if 'OR' in query.upper():
        operator = 'OR'
    elif 'NOT' in query.upper():
        operator = 'NOT'

    results = set(index[query_terms[0]]) if query_terms[0] in index else set()

    for term in query_terms[1:]:
        if term in ['AND', 'OR', 'NOT']:
            operator = term
        elif operator == 'AND' and term in index:
            results.intersection_update(index[term])
        elif operator == 'OR' and term in index:
            results.update(index[term])
        elif operator == 'NOT' and term in index:
            results.difference_update(index[term])

    return results

Ορίζει μια λειτουργία κατάταξης εγγράφων με βάση τις βαθμολογίες ομοιότητας TF-IDF.
- Χρησιμοποιεί διάνυσμα TF-IDF για να υπολογίσει τη σημασία των όρων σε κάθε έγγραφο.
- Υπολογίζει την ομοιότητα μεταξύ του ερωτήματος και των διανυσμάτων εγγράφων.
- Επιστρέφει έγγραφα ταξινομημένα με βάση τις βαθμολογίες ομοιότητας τους σε φθίνουσα σειρά.

In [8]:
def rank_by_tfidf(query_terms, documents):
    vectorizer = TfidfVectorizer()
    doc_matrix = vectorizer.fit_transform(documents)
    query_vector = vectorizer.transform([' '.join(query_terms)])
    cosine_similarities = cosine_similarity(query_vector, doc_matrix)

    ranked_docs = sorted(enumerate(cosine_similarities[0]), key=lambda x: x[1], reverse=True)
    return [(i, score) for i, score in ranked_docs if score > 0]

Ορίζει τη μηχανή αναζήτησης που συνδυάζει ερωτήματα boolean και κατάταξη TF-IDF.
- Αντιστοιχίζει την αναζήτηση του χρήστη με τίτλους εγγράφων και το περιεχόμενο.
- Επεξεργάζεται ερωτήματα τύπου boolean για συνδυασμούς λογικών όρων αναζήτησης.
- Βαθμολογεί τα αποτελέσματα με βάση τη συνάφεια χρησιμοποιώντας βαθμολογίες TF-IDF.

In [9]:
def search_engine(query, index, documents, titles):
    matching_titles = [i for i, title in enumerate(titles) if query.lower() in title.lower()]
    if matching_titles:
        for doc_id in matching_titles:
            print(f"\nTitle: {titles[doc_id]}\n{'-'*len(titles[doc_id])}\n{documents[doc_id]}\n")
        return matching_titles

    boolean_results = process_boolean_query(query, index)
    if not boolean_results:
        print("No matching documents found.")
        return []

    results = rank_by_tfidf(query.split(), [documents[i] for i in boolean_results])
    for doc_id, score in results:
        print(f"\nTitle: {titles[doc_id]}\n{'-'*len(titles[doc_id])}\n{documents[doc_id]}\n\n(Accuracy: {score * 100:.2f}%)\n")
    return results

Καθορίζει την κύρια ροή εκτέλεσης του προγράμματος.
- Προτρέπει τον χρήστη να εισέλθει σε μια κατηγορία Wikipedia για να γρατσουνίσει.
- Φέρνει σελίδες και το περιεχόμενό τους για την κατηγορία.
- Εφαρμόζει το διάνυσμα TF-IDF για την κατάταξη των σελίδων με βάση τη συνάφεια.
- Αποθηκεύει τα επεξεργασμένα δεδομένα σε ένα αρχείο CSV για μεταγενέστερη χρήση.
- Επιτρέπει στο χρήστη να αναζητήσει τα δεδομένα χρησιμοποιώντας ένα ερώτημα.
- Εμφανίζει τα αποτελέσματα αναζήτησης και τα εξάγει σε ένα αρχείο CSV.

In [10]:
category = input("Enter Wikipedia category to scrape: ")
pages = []
for term in category.split():
    pages.extend(get_pages_from_category(term, max_pages=20))
pages = list(set(pages))

documents = [get_page_content(page) for page in pages]

vectorizer = TfidfVectorizer()
doc_matrix = vectorizer.fit_transform(documents)
query_vector = vectorizer.transform([category])
cosine_similarities = cosine_similarity(query_vector, doc_matrix)

print(f"Found {len(pages)} pages in category '{category}':")
for i, page in enumerate(pages):
    print(f"{page} (Accuracy: {cosine_similarities[0][i] * 100:.2f}%)")

df = pd.DataFrame({'Title': pages, 'Content': documents})
df['Cleaned_Content'] = df['Content'].apply(preprocess_text)
inverted_index = create_inverted_index(df['Cleaned_Content'])

df.to_csv('wikipedia_data.csv', index=False)
print("Wikipedia data saved to wikipedia_data.csv")

query = input("Please select the content you want to read: ")
results = search_engine(query, inverted_index, df['Content'].tolist(), df['Title'].tolist())

if results:
    search_results_df = pd.DataFrame([
        {
            'Title': df.iloc[i]['Title'],
            'Content': df.iloc[i]['Content']
        }
        for r in results
        for i in ([r] if isinstance(r, int) else [r[0]])
    ])
    search_results_df.to_csv('search_results.csv', index=False)
    print("Search results saved to search_results.csv")


Found 20 pages in category 'war':
Mujahideen (Accuracy: 4.64%)
War (Accuracy: 29.04%)
Killed in action (Accuracy: 6.27%)
War and genocide (Accuracy: 21.30%)
Progressive war (Accuracy: 15.80%)
Philosophy of war (Accuracy: 38.67%)
Mobile Military Health Formation (Accuracy: 3.60%)
Outline of war (Accuracy: 34.53%)
Loss exchange ratio (Accuracy: 8.97%)
Coastal fortifications in Scotland (Accuracy: 4.00%)
Aggressionism (Accuracy: 9.85%)
Male warrior hypothesis (Accuracy: 1.14%)
Marketing of war (Accuracy: 13.81%)
Mercy shot (Accuracy: 0.00%)
Modern warfare (Accuracy: 10.69%)
Armed violence reduction (Accuracy: 1.94%)
Effects of war (Accuracy: 17.05%)
Civilian casualty ratio (Accuracy: 14.12%)
List of snipers (Accuracy: 0.00%)
Childhood in war (Accuracy: 25.23%)


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\sevak/nltk_data'
    - 'd:\\anaconda3\\envs\\crawler\\nltk_data'
    - 'd:\\anaconda3\\envs\\crawler\\share\\nltk_data'
    - 'd:\\anaconda3\\envs\\crawler\\lib\\nltk_data'
    - 'C:\\Users\\sevak\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
